# WorkFlow Gerencial


## 6.2 Seteo del ambiente en Google Colab

**PASO 1:** Runtime → Change Runtime Type → **Python 3**

In [1]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/.drive')

Mounted at /content/.drive


In [2]:
%%shell

mkdir -p "/content/.drive/My Drive/labo1"
mkdir -p "/content/buckets"
ln -s "/content/.drive/My Drive/labo1" /content/buckets/b1

mkdir -p ~/.kaggle
cp /content/buckets/b1/kaggle/kaggle.json  ~/.kaggle
chmod 600 ~/.kaggle/kaggle.json

mkdir -p /content/buckets/b1/exp
mkdir -p /content/buckets/b1/datasets
mkdir -p /content/datasets

webfiles="https://storage.googleapis.com/open-courses/austral2025-af91/"
destino_local="/content/datasets"
destino_bucket="/content/buckets/b1/datasets"

archivo="dataset_pequeno.csv"
if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi
if ! test -f $destino_local/$archivo; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

archivo="gerencial_competencia_2025.csv.gz"
if ! test -f $destino_bucket/$archivo; then
  wget  $webfiles/$archivo  -O $destino_bucket/$archivo
fi
if ! test -f $destino_local/$archivo; then
  cp  $destino_bucket/$archivo  $destino_local/$archivo
fi

## 6.3 Workflow

**PASO 2:** Runtime → Change Runtime Type → **R**



In [1]:
format(Sys.time(), "%a %b %d %X %Y")

[1] "Sun Nov 30 01:41:46 PM 2025"

In [2]:
# Limpiar memoria
rm(list=ls(all.names=TRUE))
gc(full=TRUE, verbose=FALSE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,660355,35.3,1454494,77.7,1454494,77.7
Vcells,1226518,9.4,8388608,64.0,1975136,15.1


In [3]:
require("data.table")

if(!require("R.utils")) install.packages("R.utils")
require("R.utils")

Loading required package: data.table

Loading required package: R.utils

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘R.utils’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘R.oo’, ‘R.methodsS3’


Loading required package: R.utils

Loading required package: R.oo

Loading required package: R.methodsS3

R.methodsS3 v1.8.2 (2022-06-13 22:00:14 UTC) successfully loaded. See ?R.methodsS3 for help.

R.oo v1.27.1 (2025-05-02 21:00:05 UTC) successfully loaded. See ?R.oo for help.


Attaching package: ‘R.oo’


The following object is masked from ‘package:R.methodsS3’:

    throw


The following objects are masked from ‘package:methods’:

    getClasses, getMethods


The following objects are masked from ‘package:base’:

    attach, detach, load, save


R.utils v2.13.0 (2025-02-24 21:20:02 UTC) successfully loaded. See ?R.utils for help.




### PARÁMETROS 

In [4]:
PARAM <- list()

# =============================================
#  CAMBIAR ESTOS VALORES 
# =============================================

# semilla prima personal 
PARAM$semilla_primigenia <- 129457  # CAMBIAR

# Número de experimento (cambiar en cada corrida)
PARAM$experimento <- 9500  # CAMBIAR 

# 5 semillas para el semillerío
PARAM$semillas_semillerio <- c(102191, 200177, 300443, 400651, 500723)  # CAMBIAR

# =============================================

PARAM$dataset <- "gerencial_competencia_2025.csv.gz"

In [5]:
# Carpeta del experimento
setwd("/content/buckets/b1/exp")
experimento_folder <- paste0("WF", PARAM$experimento)
dir.create(experimento_folder, showWarnings=FALSE)
setwd(paste0("/content/buckets/b1/exp/", experimento_folder))

cat("Trabajando en:", getwd(), "\n")

Trabajando en: /content/.drive/My Drive/labo1/exp/WF9500 


### 6.3.1 Preprocesamiento del Dataset

In [6]:
# Cargar dataset
dataset <- fread(paste0("/content/datasets/", PARAM$dataset))

cat("Dataset cargado:\n")
cat("  Filas:", nrow(dataset), "\n")
cat("  Columnas:", ncol(dataset), "\n")

Dataset cargado:
  Filas: 273666 
  Columnas: 32 


#### 6.3.1.2 CA - Catastrophe Analysis
Reparar variables rotas del mes 202006

In [7]:
# Asignar NA a variables rotas de junio 2020
dataset[foto_mes==202006, internet := NA]
dataset[foto_mes==202006, mrentabilidad := NA]
dataset[foto_mes==202006, mrentabilidad_annual := NA]
dataset[foto_mes==202006, mcomisiones := NA]
dataset[foto_mes==202006, mactivos_margen := NA]
dataset[foto_mes==202006, mpasivos_margen := NA]
dataset[foto_mes==202006, mcuentas_saldo := NA]
dataset[foto_mes==202006, ctarjeta_visa_transacciones := NA]
dataset[foto_mes==202006, mtarjeta_visa_consumo := NA]
dataset[foto_mes==202006, mtarjeta_master_consumo := NA]
dataset[foto_mes==202006, ccallcenter_transacciones := NA]
dataset[foto_mes==202006, chomebanking_transacciones := NA]

cat("Catastrophe Analysis aplicado a 202006\n")

Catastrophe Analysis aplicado a 202006


#### 6.3.1.3 FE_intra - Feature Engineering Intra-mes

##  NUEVAS VARIABLES BASADAS EN DICCIONARIO DE DATOS

Categorías:
1. **Rentabilidad**: Margen neto, eficiencia de comisiones
2. **Tarjetas**: Uso de límites, comportamiento Visa vs Master
3. **Canales**: Digital vs físico, engagement
4. **Cliente**: Valor por antigüedad, perfil VIP
5. **Salud financiera**: Ahorro vs deuda, saldo por producto

In [8]:
# Función auxiliar
atributos_presentes <- function(patributos) {
  atributos <- unique(patributos)
  comun <- intersect(atributos, colnames(dataset))
  return(length(atributos) == length(comun))
}

# =============================================
# VARIABLES ORIGINALES DEL WORKFLOW
# =============================================

# Mes del año (1 a 12) - estacionalidad
if(atributos_presentes(c("foto_mes")))
  dataset[, kmes := foto_mes %% 100]

# Variable de tesis irlandesa
if(atributos_presentes(c("mpayroll", "cliente_edad")))
  dataset[, mpayroll_sobre_edad := mpayroll / cliente_edad]

# =============================================
#  1. VARIABLES DE RENTABILIDAD Y MARGEN
# Según diccionario: mrentabilidad = ganancia total del banco
# mactivos_margen = ganancia por intereses cobrados
# mpasivos_margen = ganancia por dinero depositado
# =============================================

# Margen neto del cliente (activos - pasivos)
if(atributos_presentes(c("mactivos_margen", "mpasivos_margen")))
  dataset[, margen_neto := mactivos_margen - mpasivos_margen]

# Rentabilidad neta de comisiones
if(atributos_presentes(c("mrentabilidad", "mcomisiones")))
  dataset[, rentabilidad_sin_comisiones := mrentabilidad - mcomisiones]

# Ratio comisiones sobre rentabilidad (qué % de la ganancia son comisiones)
if(atributos_presentes(c("mcomisiones", "mrentabilidad")))
  dataset[, ratio_comisiones_rent := mcomisiones / (abs(mrentabilidad) + 1)]

# Rentabilidad mensual vs anual (consistencia)
if(atributos_presentes(c("mrentabilidad", "mrentabilidad_annual")))
  dataset[, ratio_rent_mensual_anual := mrentabilidad / (mrentabilidad_annual/12 + 1)]

cat("Variables de rentabilidad creadas\n")

# =============================================
#  2. VARIABLES DE TARJETAS (VISA + MASTER)
# Según diccionario: mlimitecompra = límite de compra (muy importante)
# mconsumototal = consumo total del mes
# =============================================

# Consumo total en tarjetas
if(atributos_presentes(c("mtarjeta_visa_consumo", "mtarjeta_master_consumo")))
  dataset[, consumo_tarjetas_total := mtarjeta_visa_consumo + mtarjeta_master_consumo]

# Ratio de uso del límite VISA (consumo/límite)
if(atributos_presentes(c("Visa_mconsumototal", "Visa_mlimitecompra")))
  dataset[, visa_ratio_uso_limite := Visa_mconsumototal / (Visa_mlimitecompra + 1)]

# Ratio de uso del límite MASTER
if(atributos_presentes(c("Master_mconsumototal", "Master_mlimitecompra")))
  dataset[, master_ratio_uso_limite := Master_mconsumototal / (Master_mlimitecompra + 1)]

# Límite total de tarjetas
if(atributos_presentes(c("Visa_mlimitecompra", "Master_mlimitecompra")))
  dataset[, limite_tarjetas_total := Visa_mlimitecompra + Master_mlimitecompra]

# Ratio Visa vs Master (preferencia de tarjeta)
if(atributos_presentes(c("Visa_mconsumototal", "Master_mconsumototal")))
  dataset[, ratio_visa_master := Visa_mconsumototal / (Master_mconsumototal + 1)]

# Cantidad total de consumos con tarjeta
if(atributos_presentes(c("Visa_cconsumos", "Master_cconsumos")))
  dataset[, cantidad_consumos_tarjeta := Visa_cconsumos + Master_cconsumos]

# Ticket promedio por consumo de tarjeta
if(atributos_presentes(c("Visa_mconsumototal", "Master_mconsumototal", "Visa_cconsumos", "Master_cconsumos")))
  dataset[, ticket_promedio_tarjeta := (Visa_mconsumototal + Master_mconsumototal) / (Visa_cconsumos + Master_cconsumos + 1)]

# Saldo total en tarjetas
if(atributos_presentes(c("Visa_msaldototal", "Master_msaldototal")))
  dataset[, saldo_tarjetas_total := Visa_msaldototal + Master_msaldototal]

# Ratio saldo/límite (nivel de endeudamiento en tarjetas)
if(atributos_presentes(c("Visa_msaldototal", "Master_msaldototal", "Visa_mlimitecompra", "Master_mlimitecompra")))
  dataset[, ratio_saldo_limite_tarjetas := (Visa_msaldototal + Master_msaldototal) / (Visa_mlimitecompra + Master_mlimitecompra + 1)]

cat("Variables de tarjetas creadas\n")

# =============================================
#  3. VARIABLES DE CANALES (DIGITAL VS FÍSICO)
# Según diccionario: chomebanking_transacciones = transacciones digitales
# ccallcenter_transacciones = canal telefónico
# ctrx_quarter = movimientos voluntarios últimos 90 días
# =============================================

# Ratio digital vs telefónico
if(atributos_presentes(c("chomebanking_transacciones", "ccallcenter_transacciones")))
  dataset[, ratio_digital_telefono := chomebanking_transacciones / (ccallcenter_transacciones + 1)]

# Actividad trimestral promedio mensual
if(atributos_presentes(c("ctrx_quarter")))
  dataset[, actividad_mensual_promedio := ctrx_quarter / 3]

# Intensidad de uso digital (transacciones por producto)
if(atributos_presentes(c("chomebanking_transacciones", "cproductos")))
  dataset[, intensidad_digital := chomebanking_transacciones / (cproductos + 1)]

cat("Variables de canales creadas\n")

# =============================================
#  4. VARIABLES DE CLIENTE (VALOR Y PERFIL)
# Según diccionario: cliente_antiguedad = meses como cliente
# cliente_vip = si marketing lo considera VIP (<2% de premium)
# =============================================

# Valor por antigüedad (rentabilidad * años de cliente)
if(atributos_presentes(c("mrentabilidad", "cliente_antiguedad")))
  dataset[, valor_por_antiguedad := mrentabilidad * (cliente_antiguedad / 12)]

# Rentabilidad por año de relación
if(atributos_presentes(c("mrentabilidad_annual", "cliente_antiguedad")))
  dataset[, rent_por_anio_relacion := mrentabilidad_annual / (cliente_antiguedad / 12 + 1)]

# VIP rentable (es VIP y además es rentable)
if(atributos_presentes(c("cliente_vip", "mrentabilidad")))
  dataset[, vip_rentable := cliente_vip * mrentabilidad]

# Productos por año de antigüedad
if(atributos_presentes(c("cproductos", "cliente_antiguedad")))
  dataset[, productos_por_anio := cproductos / (cliente_antiguedad / 12 + 1)]

# Cliente joven digital (edad < 35 y usa homebanking)
if(atributos_presentes(c("cliente_edad", "internet")))
  dataset[, joven_digital := as.integer(cliente_edad < 35 & internet == 1)]

cat("Variables de cliente creadas\n")

# =============================================
# 5. VARIABLES DE SALUD FINANCIERA
# Según diccionario: mcaja_ahorro = ahorro
# mprestamos_personales = deuda
# mcuentas_saldo = saldo total
# =============================================

# Ratio ahorro vs deuda (salud financiera)
if(atributos_presentes(c("mcaja_ahorro", "mprestamos_personales")))
  dataset[, ratio_ahorro_deuda := mcaja_ahorro / (mprestamos_personales + 1)]

# Saldo por producto (diversificación)
if(atributos_presentes(c("mcuentas_saldo", "cproductos")))
  dataset[, saldo_por_producto := mcuentas_saldo / (cproductos + 1)]

# Capacidad de pago (payroll vs deuda en tarjetas)
if(atributos_presentes(c("mpayroll", "Visa_msaldototal", "Master_msaldototal")))
  dataset[, capacidad_pago := mpayroll / (Visa_msaldototal + Master_msaldototal + 1)]

# Ratio payroll vs rentabilidad (dependencia del sueldo)
if(atributos_presentes(c("mpayroll", "mrentabilidad")))
  dataset[, ratio_payroll_rent := mpayroll / (abs(mrentabilidad) + 1)]

# Cuenta corriente + caja ahorro (liquidez total)
if(atributos_presentes(c("mcuenta_corriente", "mcaja_ahorro")))
  dataset[, liquidez_total := mcuenta_corriente + mcaja_ahorro]

# Ratio liquidez vs deuda tarjetas
if(atributos_presentes(c("mcuenta_corriente", "mcaja_ahorro", "Visa_msaldototal", "Master_msaldototal")))
  dataset[, ratio_liquidez_deuda_tarjetas := (mcuenta_corriente + mcaja_ahorro) / (Visa_msaldototal + Master_msaldototal + 1)]

cat("Variables de salud financiera creadas\n")

# =============================================
#  6. VARIABLES DE TRANSFERENCIAS Y PAGOS
# =============================================

# Ratio transferencias recibidas vs rentabilidad
if(atributos_presentes(c("mtransferencias_recibidas", "mrentabilidad")))
  dataset[, ratio_transf_rent := mtransferencias_recibidas / (abs(mrentabilidad) + 1)]

# Débitos automáticos en Visa (nivel de automatización)
if(atributos_presentes(c("mtarjeta_visa_debitos_automaticos", "Visa_mconsumototal")))
  dataset[, ratio_debitos_auto_visa := mtarjeta_visa_debitos_automaticos / (Visa_mconsumototal + 1)]

cat("Variables de transferencias creadas\n")

# =============================================
#  7. VARIABLES DE COMISIONES
# =============================================

# Comisiones totales
if(atributos_presentes(c("mcomisiones_mantenimiento", "mcomisiones_otras")))
  dataset[, comisiones_totales := mcomisiones_mantenimiento + mcomisiones_otras]

# Ratio comisiones mantenimiento vs otras
if(atributos_presentes(c("mcomisiones_mantenimiento", "mcomisiones_otras")))
  dataset[, ratio_comisiones_mant_otras := mcomisiones_mantenimiento / (mcomisiones_otras + 1)]

cat("\n========================================\n")
cat("Feature Engineering intra-mes completado\n")
cat("Columnas actuales:", ncol(dataset), "\n")
cat("======================================== \n")

Variables de rentabilidad creadas
Variables de tarjetas creadas
Variables de canales creadas
Variables de cliente creadas
Variables de salud financiera creadas
Variables de transferencias creadas

Feature Engineering intra-mes completado
Columnas actuales: 50 


#### 6.3.1.5 FEhist - Feature Engineering Histórico

**Esta es la etapa que más aporta a la ganancia**

In [9]:
cat("Iniciando Feature Engineering Histórico...\n")

# Columnas que se pueden laguear
cols_lagueables <- copy(setdiff(
  colnames(dataset),
  c("numero_de_cliente", "foto_mes", "clase_ternaria")
))

cat("  Variables a procesar:", length(cols_lagueables), "\n")

# Lags de orden 1
cat("  Creando lags orden 1...\n")
dataset[,
  paste0(cols_lagueables, "_lag1") := shift(.SD, 1, NA, "lag"),
  by = numero_de_cliente,
  .SDcols = cols_lagueables
]

# Lags de orden 2
cat("  Creando lags orden 2...\n")
dataset[,
  paste0(cols_lagueables, "_lag2") := shift(.SD, 2, NA, "lag"),
  by = numero_de_cliente,
  .SDcols = cols_lagueables
]

# Delta lags
cat("  Creando deltas...\n")
for (vcol in cols_lagueables) {
  dataset[, paste0(vcol, "_delta1") := get(vcol) - get(paste0(vcol, "_lag1"))]
  dataset[, paste0(vcol, "_delta2") := get(vcol) - get(paste0(vcol, "_lag2"))]
}

cat("Lags y deltas completados. Columnas:", ncol(dataset), "\n")

Iniciando Feature Engineering Histórico...
  Variables a procesar: 47 
  Creando lags orden 1...
  Creando lags orden 2...
  Creando deltas...
Lags y deltas completados. Columnas: 238 


####  TREND Features - Tendencias

**Según experimentos: las TREND son el CORE del modelo**

In [10]:
cat("Creando TREND features...\n")

# Tendencia: promedio de cambio entre períodos
for (vcol in cols_lagueables) {
  dataset[, paste0(vcol, "_trend2") :=
    (get(vcol) - get(paste0(vcol, "_lag2"))) / 2]
}

# Aceleración: cambio del cambio
for (vcol in cols_lagueables) {
  dataset[, paste0(vcol, "_accel") :=
    get(paste0(vcol, "_delta1")) - get(paste0(vcol, "_delta2"))]
}

cat("TREND features completadas. Columnas totales:", ncol(dataset), "\n")

Creando TREND features...
TREND features completadas. Columnas totales: 332 


In [11]:
# Verificar columnas creadas
cat("\nResumen de columnas:\n")
cat("  Total:", ncol(dataset), "\n")
cat("  Originales + FE intra: ~60\n")
cat("  + Lags (x2):", length(cols_lagueables) * 2, "\n")
cat("  + Deltas (x2):", length(cols_lagueables) * 2, "\n")
cat("  + Trends (x2):", length(cols_lagueables) * 2, "\n")


Resumen de columnas:
  Total: 332 
  Originales + FE intra: ~60
  + Lags (x2): 94 
  + Deltas (x2): 94 
  + Trends (x2): 94 


### 6.3.2 Modelado

#### 6.3.2.1 Training Strategy

In [12]:
# Meses para validación y entrenamiento
PARAM$trainingstrategy$validate <- c(202107)

PARAM$trainingstrategy$training <- c(
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

# Sin undersampling para Gerencial
PARAM$trainingstrategy$training_pct <- 1.0

# Clase positiva
PARAM$trainingstrategy$positivos <- c("BAJA+1", "BAJA+2")

cat("Training Strategy configurado\n")
cat("  Meses training:", length(PARAM$trainingstrategy$training), "\n")
cat("  Mes validación:", PARAM$trainingstrategy$validate, "\n")

Training Strategy configurado
  Meses training: 14 
  Mes validación: 202107 


In [13]:
# Crear clase binaria
dataset[, clase01 := ifelse(clase_ternaria %in% PARAM$trainingstrategy$positivos, 1, 0)]

# Campos para entrenar (quitar clase y auxiliares)
campos_buenos <- copy(setdiff(
  colnames(dataset),
  c("clase_ternaria", "clase01", "azar")
))

cat("Campos para entrenar:", length(campos_buenos), "\n")

Campos para entrenar: 331 


In [14]:
# Preparar para undersampling (aunque no lo usemos)
set.seed(PARAM$semilla_primigenia, kind = "L'Ecuyer-CMRG")
dataset[, azar := runif(nrow(dataset))]

# Marcar registros de training
dataset[, fold_train := foto_mes %in% PARAM$trainingstrategy$training &
  (clase_ternaria %in% c("BAJA+1", "BAJA+2") |
   azar < PARAM$trainingstrategy$training_pct)]

# Instalar LightGBM
if(!require("lightgbm")) install.packages("lightgbm")
require("lightgbm")

# Dataset de training
dtrain <- lgb.Dataset(
  data = data.matrix(dataset[fold_train == TRUE, campos_buenos, with = FALSE]),
  label = dataset[fold_train == TRUE, clase01],
  free_raw_data = TRUE
)

cat("dtrain creado con", nrow(dtrain), "registros\n")

Loading required package: lightgbm

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘lightgbm’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: lightgbm



dtrain creado con 224011 registros


In [15]:
# Dataset de validación
dvalidate <- lgb.Dataset(
  data = data.matrix(dataset[foto_mes %in% PARAM$trainingstrategy$validate, campos_buenos, with = FALSE]),
  label = dataset[foto_mes %in% PARAM$trainingstrategy$validate, clase01],
  free_raw_data = TRUE
)

cat("dvalidate creado con", nrow(dvalidate), "registros\n")

dvalidate creado con 16551 registros


#### 6.3.2.2 Hyperparameter Tuning

** Parámetros OPTIMIZADOS según profesor:**
- learning_rate = 0.01 (más lento, más preciso)
- early_stopping_rounds = 1000 (no cortar prematuramente)
- num_iterations = 9999 (permitir más aprendizaje)
- 50 iteraciones de Bayesian Optimization

In [16]:
# Paquetes para Bayesian Optimization
if(!require("DiceKriging")) install.packages("DiceKriging")
require("DiceKriging")

if(!require("mlrMBO")) install.packages("mlrMBO")
require("mlrMBO")

Loading required package: DiceKriging

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘DiceKriging’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

Loading required package: DiceKriging

Loading required package: mlrMBO

Warning message in library(package, lib.loc = lib.loc, character.only = TRUE, logical.return = TRUE, :
“there is no package called ‘mlrMBO’”
Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘XML’, ‘fastmatch’, ‘RcppArmadillo’, ‘mlr’, ‘ParamHelpers’, ‘smoof’, ‘BBmisc’, ‘checkmate’, ‘lhs’, ‘parallelMap’


Loading required package: mlrMBO

Loading required package: mlr

Loading required package: ParamHelpers


Attaching package: ‘ParamHelpers’


The following object is masked from ‘package:R.utils’:

    isVector



Attaching package: ‘mlr’


The following objects are masked from ‘package

In [17]:
# =============================================
# PARÁMETROS OPTIMIZADOS
# =============================================

# Iteraciones de Bayesian Optimization
PARAM$hipeparametertuning$num_interations <- 50  # ANTES: 10

# Parámetros FIJOS de LightGBM
PARAM$lgbm$param_fijos <- list(
  objective = "binary",
  metric = "auc",
  first_metric_only = TRUE,
  boost_from_average = TRUE,
  feature_pre_filter = FALSE,
  verbosity = -100,
  force_row_wise = TRUE,
  seed = PARAM$semilla_primigenia,
  max_bin = 31,

  # CAMBIOS CLAVE DEL PROFESOR:
  learning_rate = 0.01,          # ANTES: 0.03
  num_iterations = 9999,         # ANTES: 2048
  early_stopping_rounds = 1000   # ANTES: 200

  # feature_fraction se optimiza en BO
)

# Hiperparámetros a OPTIMIZAR
PARAM$hipeparametertuning$hs <- makeParamSet(
  makeIntegerParam("num_leaves", lower = 8L, upper = 256L),
  makeIntegerParam("min_data_in_leaf", lower = 8L, upper = 4096L),
  makeNumericParam("feature_fraction", lower = 0.3, upper = 0.9),
  makeNumericParam("bagging_fraction", lower = 0.5, upper = 1.0),
  makeIntegerParam("bagging_freq", lower = 1L, upper = 5L)
)

cat("Parámetros configurados:\n")
cat("  learning_rate:", PARAM$lgbm$param_fijos$learning_rate, "\n")
cat("  early_stopping_rounds:", PARAM$lgbm$param_fijos$early_stopping_rounds, "\n")
cat("  num_iterations:", PARAM$lgbm$param_fijos$num_iterations, "\n")
cat("  Iteraciones BO:", PARAM$hipeparametertuning$num_interations, "\n")

Parámetros configurados:
  learning_rate: 0.01 
  early_stopping_rounds: 1000 
  num_iterations: 9999 
  Iteraciones BO: 50 


In [18]:
# Función objetivo para Bayesian Optimization
EstimarGanancia_AUC_lightgbm <- function(x) {

  # Combinar parámetros fijos con los que se optimizan
  param_completo <- modifyList(PARAM$lgbm$param_fijos, x)

  # Entrenar modelo
  modelo_train <- lgb.train(
    data = dtrain,
    valids = list(valid = dvalidate),
    eval = "auc",
    param = param_completo,
    verbose = -100
  )

  # Obtener AUC en validación
  AUC <- modelo_train$record_evals$valid$auc$eval[[modelo_train$best_iter]]

  # Guardar número de iteraciones óptimo
  attr(AUC, "extras") <- list("num_iterations" = modelo_train$best_iter)

  # Liberar memoria
  rm(modelo_train)
  gc(full = TRUE, verbose = FALSE)

  message(format(Sys.time(), "%H:%M:%S"), " AUC: ", round(AUC, 6))

  return(AUC)
}

In [19]:
# Configurar Bayesian Optimization
configureMlr(show.learner.output = FALSE)

obj.fun <- makeSingleObjectiveFunction(
  fn = EstimarGanancia_AUC_lightgbm,
  minimize = FALSE,  # Maximizar AUC
  noisy = FALSE,
  par.set = PARAM$hipeparametertuning$hs,
  has.simple.signature = FALSE
)

# Guardar cada 600 segundos
ctrl <- makeMBOControl(
  save.on.disk.at.time = 600,
  save.file.path = "HT.RDATA"
)

ctrl <- setMBOControlTermination(
  ctrl,
  iters = PARAM$hipeparametertuning$num_interations
)

ctrl <- setMBOControlInfill(ctrl, crit = makeMBOInfillCritEI())

surr.km <- makeLearner(
  "regr.km",
  predict.type = "se",
  covtype = "matern3_2",
  control = list(trace = TRUE)
)

cat("Bayesian Optimization configurada\n")

Bayesian Optimization configurada


In [20]:
# =============================================
# 🚀 CORRER BAYESIAN OPTIMIZATION
# =============================================
# Tiempo estimado: 2-3 horas para 50 iteraciones

cat("\n========================================\n")
cat("INICIANDO BAYESIAN OPTIMIZATION\n")
cat("Iteraciones:", PARAM$hipeparametertuning$num_interations, "\n")
cat("Inicio:", format(Sys.time(), "%H:%M:%S"), "\n")
cat("========================================\n\n")

if (!file.exists("HT.RDATA")) {
  bayesiana_salida <- mbo(obj.fun, learner = surr.km, control = ctrl)
} else {
  cat("Retomando desde HT.RDATA...\n")
  bayesiana_salida <- mboContinue("HT.RDATA")
}

cat("\n========================================\n")
cat("BAYESIAN OPTIMIZATION COMPLETADA\n")
cat("Fin:", format(Sys.time(), "%H:%M:%S"), "\n")
cat("========================================\n")


INICIANDO BAYESIAN OPTIMIZATION
Iteraciones: 50 
Inicio: 13:57:01 

Retomando desde HT.RDATA...


14:10:18 AUC: 0.934099

[mbo] 33: num_leaves=197; min_data_in_leaf=10; feature_fraction=0.736; bagging_fraction=0.933; bagging_freq=1 : y = 0.934 : 794.7 secs : infill_ei

Saved the current state after iteration 34 in the file HT.RDATA.

14:25:52 AUC: 0.936005

[mbo] 34: num_leaves=254; min_data_in_leaf=65; feature_fraction=0.9; bagging_fraction=0.875; bagging_freq=2 : y = 0.936 : 933.2 secs : infill_ei

Saved the current state after iteration 35 in the file HT.RDATA.

14:40:45 AUC: 0.935848

[mbo] 35: num_leaves=254; min_data_in_leaf=47; feature_fraction=0.764; bagging_fraction=0.854; bagging_freq=3 : y = 0.936 : 892.8 secs : infill_ei

Saved the current state after iteration 36 in the file HT.RDATA.

15:00:03 AUC: 0.93557

[mbo] 36: num_leaves=256; min_data_in_leaf=132; feature_fraction=0.551; bagging_fraction=0.881; bagging_freq=2 : y = 0.936 : 1156.9 secs : infill_ei

Saved the current state after iteration 37 in the file HT.RDATA.

15:11:41 AUC: 0.936211

[mbo] 37: num_leaves=216;


BAYESIAN OPTIMIZATION COMPLETADA
Fin: 18:16:25 


In [21]:
# Extraer mejores hiperparámetros
tb_bayesiana <- as.data.table(bayesiana_salida$opt.path)
setorder(tb_bayesiana, -y, -num_iterations)

# Guardar log
fwrite(tb_bayesiana, file = "BO_log.txt", sep = "\t")

# Mejores hiperparámetros
PARAM$out$lgbm$mejores_hiperparametros <- tb_bayesiana[
  1,
  setdiff(colnames(tb_bayesiana),
    c("y", "dob", "eol", "error.message", "exec.time", "ei", "error.model",
      "train.time", "prop.type", "propose.time", "se", "mean", "iter")),
  with = FALSE
]

cat("\n🏆 MEJORES HIPERPARÁMETROS ENCONTRADOS:\n")
print(PARAM$out$lgbm$mejores_hiperparametros)
cat("\nMejor AUC:", tb_bayesiana[1, y], "\n")


 MEJORES HIPERPARÁMETROS ENCONTRADOS:
   num_leaves min_data_in_leaf feature_fraction bagging_fraction bagging_freq
        <int>            <int>            <num>            <num>        <int>
1:        231               19          0.83072        0.8602651            4
   num_iterations
            <int>
1:           1009

Mejor AUC: 0.9374576 


### 6.3.3 Producción - Final Training con Semillerío

In [22]:
# Meses para entrenamiento final (incluye validación)
PARAM$trainingstrategy$final_train <- c(
  202107,
  202106, 202105, 202104, 202103, 202102, 202101,
  202012, 202011, 202010, 202009, 202008, 202007,
  202006, 202005
)

dataset[, fold_final_train := foto_mes %in% PARAM$trainingstrategy$final_train]

# Dataset final de training
dfinal_train <- lgb.Dataset(
  data = data.matrix(dataset[fold_final_train == TRUE, campos_buenos, with = FALSE]),
  label = dataset[fold_final_train == TRUE, clase01],
  free_raw_data = TRUE
)

cat("dfinal_train:", nrow(dfinal_train), "registros\n")

dfinal_train: 240562 registros


In [23]:
# Combinar parámetros fijos con los mejores encontrados
fijos <- copy(PARAM$lgbm$param_fijos)
fijos$num_iterations <- NULL
fijos$early_stopping_rounds <- NULL

param_final <- c(fijos, PARAM$out$lgbm$mejores_hiperparametros)

cat("Parámetros finales preparados\n")

Parámetros finales preparados


In [24]:
# Preparar datos del futuro para predicción
PARAM$trainingstrategy$future <- c(202109)
dfuture <- dataset[foto_mes %in% PARAM$trainingstrategy$future]

cat("dfuture:", nrow(dfuture), "registros a predecir\n")

dfuture: 16551 registros a predecir


####  SEMILLERÍO - Ensemble de Múltiples Modelos

Entrenar 5 modelos con diferentes semillas y promediar predicciones

In [25]:
# =============================================
#  SEMILLERÍO - 5 MODELOS
# =============================================

cat("\n========================================\n")
cat("INICIANDO SEMILLERÍO\n")
cat("Semillas:", paste(PARAM$semillas_semillerio, collapse = ", "), "\n")
cat("========================================\n\n")

# Lista para guardar predicciones
lista_predicciones <- list()

for(i in seq_along(PARAM$semillas_semillerio)) {

  semilla_actual <- PARAM$semillas_semillerio[i]
  cat("Modelo", i, "/", length(PARAM$semillas_semillerio), "- Semilla:", semilla_actual, "\n")

  # Modificar semilla en parámetros
  param_semilla <- param_final
  param_semilla$seed <- semilla_actual

  # Entrenar modelo
  modelo_i <- lgb.train(
    data = dfinal_train,
    param = param_semilla,
    verbose = -100
  )

  # Predecir
  pred_i <- predict(
    modelo_i,
    data.matrix(dfuture[, campos_buenos, with = FALSE])
  )

  lista_predicciones[[i]] <- pred_i

  cat("  ✓ Completado\n")

  # Liberar memoria
  rm(modelo_i)
  gc(verbose = FALSE)
}

# Promediar predicciones
prediccion <- Reduce(`+`, lista_predicciones) / length(lista_predicciones)

cat("\n========================================\n")
cat("SEMILLERÍO COMPLETADO\n")
cat("Predicción promediada de", length(PARAM$semillas_semillerio), "modelos\n")
cat("========================================\n")


INICIANDO SEMILLERÍO
Semillas: 102191, 200177, 300443, 400651, 500723 

Modelo 1 / 5 - Semilla: 102191 
  ✓ Completado
Modelo 2 / 5 - Semilla: 200177 
  ✓ Completado
Modelo 3 / 5 - Semilla: 300443 
  ✓ Completado
Modelo 4 / 5 - Semilla: 400651 
  ✓ Completado
Modelo 5 / 5 - Semilla: 500723 
  ✓ Completado

SEMILLERÍO COMPLETADO
Predicción promediada de 5 modelos


In [26]:
# Crear tabla de predicciones
tb_prediccion <- dfuture[, list(numero_de_cliente)]
tb_prediccion[, prob := prediccion]

# Guardar predicciones
fwrite(tb_prediccion, file = "prediccion.txt", sep = "\t")

cat("Predicciones guardadas en prediccion.txt\n")

Predicciones guardadas en prediccion.txt


### 6.3.4 Submit a Kaggle

In [27]:
# =============================================
# 📤 GENERAR Y SUBIR A KAGGLE
# =============================================

PARAM$kaggle$competencia <- "labo-i-2025-virtual-gerencial"
PARAM$kaggle$cortes <- seq(800, 1300, by = 25)  # Más granularidad

# Ordenar por probabilidad descendente
setorder(tb_prediccion, -prob)

dir.create("kaggle", showWarnings = FALSE)

cat("\n========================================\n")
cat("SUBIENDO A KAGGLE\n")
cat("Cortes:", min(PARAM$kaggle$cortes), "a", max(PARAM$kaggle$cortes), "\n")
cat("========================================\n\n")

for (envios in PARAM$kaggle$cortes) {

  tb_prediccion[, Predicted := 0L]
  tb_prediccion[1:envios, Predicted := 1L]

  archivo_kaggle <- paste0("./kaggle/KA", PARAM$experimento, "_", envios, ".csv")

  fwrite(
    tb_prediccion[, list(numero_de_cliente, Predicted)],
    file = archivo_kaggle,
    sep = ","
  )

  # Submit a Kaggle
  comando <- "kaggle competitions submit"
  competencia <- paste("-c", PARAM$kaggle$competencia)
  arch <- paste("-f", archivo_kaggle)
  mensaje <- paste0("-m 'FE_dict envios=", envios, " exp=", PARAM$experimento, "'")

  linea <- paste(comando, competencia, arch, mensaje)
  salida <- system(linea, intern = TRUE)

  cat("Corte", envios, "- subido\n")
}

cat("\n TODOS LOS SUBMITS COMPLETADOS\n")


SUBIENDO A KAGGLE
Cortes: 800 a 1300 

Corte 800 - subido
Corte 825 - subido
Corte 850 - subido
Corte 875 - subido
Corte 900 - subido
Corte 925 - subido
Corte 950 - subido
Corte 975 - subido
Corte 1000 - subido
Corte 1025 - subido
Corte 1050 - subido
Corte 1075 - subido
Corte 1100 - subido
Corte 1125 - subido
Corte 1150 - subido
Corte 1175 - subido
Corte 1200 - subido
Corte 1225 - subido


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA9500_1250.csv -m 'FE_dict envios=1250 exp=9500'' had status 1”


Corte 1250 - subido


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA9500_1275.csv -m 'FE_dict envios=1275 exp=9500'' had status 1”


Corte 1275 - subido


Warning message in system(linea, intern = TRUE):
“running command 'kaggle competitions submit -c labo-i-2025-virtual-gerencial -f ./kaggle/KA9500_1300.csv -m 'FE_dict envios=1300 exp=9500'' had status 1”


Corte 1300 - subido

 TODOS LOS SUBMITS COMPLETADOS


In [28]:
# Guardar parámetros
if(!require("yaml")) install.packages("yaml")
require("yaml")

write_yaml(PARAM, file = "PARAM.yml")

cat("Parámetros guardados en PARAM.yml\n")

Loading required package: yaml



Parámetros guardados en PARAM.yml


In [29]:
cat("\n========================================\n")
cat(" WORKFLOW COMPLETADO\n")
cat("Fin:", format(Sys.time(), "%a %b %d %X %Y"), "\n")
cat("========================================\n")


🏁 WORKFLOW COMPLETADO
Fin: Sun Nov 30 06:58:02 PM 2025 
